In [ ]:
import os

<img src="../../img/python-logo-no-text.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;">
  <b>Dynamic Code Execution</b>
</div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<br/>
<div style="text-align:center;">module_220_advanced_topics/topic_200_dynamic_code_execution</div>


# Dynamic Code Execution

Dynamic access to attributes

In [ ]:
from dataclasses import dataclass

In [ ]:
@dataclass
class MyClass:
    x: int

In [ ]:
my_obj = MyClass(1, 2)
my_obj.x

In [ ]:
hasattr(my_obj, "x")

In [ ]:
getattr(my_obj, "x")

In [ ]:
hasattr(my_obj, "z")

# Dynamically created classes

In [ ]:
class Bar:
    pass

In [ ]:
my_bar = Bar()
my_bar

In [ ]:
type(my_bar).mro()

In [ ]:
Foo = type("Foo", (object,), {})

In [ ]:
my_foo = Foo()
my_foo

In [ ]:
type(my_foo).mro()

In [ ]:
def make_class(name):
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __repr__(self):
        slot_value_repr = ", ".join(
            f"{slot}={value!r}" for slot, value in self.__dict__.items()
        )
        return f"{type(self).__name__}({slot_value_repr})"

    class_attributes = dict(__init__=__init__, __repr__=__repr__)

    return type(name, (object,), class_attributes)

In [ ]:
MyClass1 = make_class("MyClass1")

In [ ]:
my_obj = MyClass1()
my_obj

In [ ]:
MyClass2 = make_class("MyClass2")

In [ ]:
my_obj = MyClass2(x=1, y=2)
my_obj


## Mini-workshop "Programmatic Definition of Classes"

Write a function `make_classes(specs: list)` that takes a list of class names and
returns a list of corresponding classes whose constructor takes two arguments,
initializing `x` and `y` attributes of the object. The resulting classes should have a
`__repr__()` method that represents them in a human-readable form and a `move(dx, dy)
method that moves `x` and `y` by `dx` and `dy`, respectively.

In [ ]:
def make_classes(names):
    def make_one_class(name):
        def __init__(self, x, y):
            self.x = x
            self.y = y

        def __repr__(self):
            return f"{type(self).__name__}(x={self.x!r},y={self.y!r})"

        def move(self, dx, dy):
            self.x += dx
            self.y += dy

        class_attributes = dict(__init__=__init__, __repr__=__repr__, move=move)

        return type(name, (object,), class_attributes)

    return [make_one_class(name) for name in names]

In [ ]:
C1, C2, C3 = make_classes(["C1", "C2", "C3"])

In [ ]:
c1 = C1(1, 2)
assert c1.__repr__() == "C1(x=1,y=2)"
c1.move(3, 4)
assert c1.__repr__() == "C1(x=4,y=6)"

In [ ]:
c2 = C2(4, 5)
assert c2.__repr__() == "C2(x=4,y=5)"
c2.move(-1, -1)
assert c2.__repr__() == "C2(x=3,y=4)"

In [ ]:
c3 = C3(11, 22)
assert c3.__repr__() == "C3(x=11,y=22)"
c3.move(0, 0)
assert c3.__repr__() == "C3(x=11,y=22)"

Dynamic code execution

In [ ]:
eval("1 + 2")

In [ ]:
x = 1
eval("x + 1")

In [ ]:
def f(x, y):
    return 10 * x + y

In [ ]:
eval("f(2, 3)")

In [ ]:
# eval("f(2, 3)", {})

In [ ]:
eval("g(2, 3)", {"g": f})

In [ ]:
eval("g.__name__", {"g": f})

In [ ]:
my_src = """
def my_fun(x):
    print(f"my_fun({x})")
    return x ** 3
"""

In [ ]:
exec(my_src)

In [ ]:
my_fun(2)

In [ ]:
def build_power_fun(name, power):
    return f"def {name}(n):\n    return n ** {power}\n"

In [ ]:
exec(build_power_fun("a_square", 2))

In [ ]:
a_square(4)

In [ ]:
my_globals = {}
exec(build_power_fun("pow10", 10), my_globals)

In [ ]:
# pow10(2)

In [ ]:
my_globals.get("pow10")

In [ ]:
my_globals["pow10"](2)

In [ ]:
# math.pow(2, 10)

In [ ]:
exec("import math")

In [ ]:
math.pow(2, 10)

In [ ]:
my_code = compile(build_power_fun("foo", 3), "<string>", "exec")

In [ ]:
exec(my_code)

In [ ]:
foo(2)


## Mini-workshop "Executable Configuration"

Write a function `process_config(path)` that reads in a file from `path` and
executes the code in the file.

In [ ]:
def process_config(path):
    with open(path, "r", encoding="utf-8") as file:
        src = file.read()
    exec(src)

In [ ]:
# from pathlib import Path
# process_config(Path.home() / "Tmp/my_config.py")